In [ ]:
import pandas as pd
import json
import ast
from pandas.io import gbq
import os

In [ ]:
files = os.listdir('./data')

In [ ]:
text = []
count = 0
for i in files:
    f = f'{count} = {i}/'
    text.append(f)
    count += 1
text_str = ''.join(text).replace('/','\n')

In [58]:
df = pd.read_csv(f'./data/juan_gabriel.csv')

In [ ]:
df.sample(20)

In [ ]:
df['album'] = df['album'].apply(lambda x: ast.literal_eval(x))

In [ ]:
def unnest_values(column):
    n = 0
    a_values = []  
    for row in column:
        new_value = {'key':n}
        row.update(new_value)
        a_values.append(row)
        n += 1
    nest_df = pd.DataFrame(a_values)
    return nest_df

df = df.merge(unnest_values(df['album']), on='key',how='left')
df.columns = df.columns.str.replace(r'_x','_track')
df.columns = df.columns.str.replace(r'_y','_album')

In [ ]:
df['artists_track'] = df['artists_track'].apply(lambda x: ast.literal_eval(x))
df['artists_track'] = df['artists_track'].apply(lambda x: x[0])

In [ ]:
df.sample(5)

In [ ]:
#df['images'] = df['images'].apply(lambda x: ast.literal_eval(x))
df['images'] = df['images'].apply(lambda x: x[0])

In [ ]:
df = df.merge(unnest_values(df['images']), on='key',how='left')
df = df.merge(unnest_values(df['artists_track']), on='key',how='left')

In [ ]:
df.columns

In [ ]:
df

In [ ]:
df.drop(['album','external_ids','artists_track','width','height','images','external_urls_track','external_urls_album','artists_album','external_ids','href_track','href_album','uri_album','key','type_album','type_track','type','uri'],axis=1,inplace=True)

In [ ]:
df.rename(columns={'url': 'image_url'},inplace=True)

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df = df[[
'artist',
'id_track',
'name_track',
'name',
'duration_ms',
'explicit',
'popularity',
'preview_url',
'id_album',
'name_album',
'release_date',
'total_tracks',
'album_type',
'image_url'
]]

In [ ]:
df.rename(columns={'name':'author'})

In [ ]:
df.sample(5)

In [ ]:
df[['duration_ms','popularity','total_tracks']] = df[['duration_ms','popularity','total_tracks']].astype(int)

In [ ]:
df['release_date'] = pd.to_datetime(df['release_date'])

In [ ]:
df.to_gbq(destination_table='spotify.songs',
project_id='general-project-352815',
if_exists='append')